In [1]:
from pds4_tools import pds4_read       # to read and inspect the data and metadata
import matplotlib.pyplot as plt        # for plotting
import matplotlib
import numpy as np
from PIL import Image
# for plotting in Jupyter notebooks
%matplotlib notebook

from skimage import exposure
from skimage import data, img_as_float
import colour
from colour.plotting import *
import glob

from colour_demosaicing import (
    demosaicing_CFA_Bayer_bilinear,
    demosaicing_CFA_Bayer_Malvar2004,
    demosaicing_CFA_Bayer_Menon2007,
    mosaicing_CFA_Bayer)

cctf_encoding = colour.cctf_encoding
colour.utilities.filter_warnings()
# colour.utilities.describe_environment();

def read_pds(path):
    data = pds4_read(path, quiet=True)
    img = np.array(data[0].data)
    img = img_as_float(img)
    return img
def debayer_img(img, CFA='RGGB'):
    # Menon2007 yields better edges than bilinear
    debayered = cctf_encoding(demosaicing_CFA_Bayer_Menon2007(img, CFA))
    return debayered
def stretch_img(img):
    p2, p98 = np.percentile(img, (2, 98))
    img = exposure.rescale_intensity(img, in_range=(p2, p98))
    return img
def export_img(name, img):
    pil_img = Image.fromarray(np.uint8(img*255))
    pil_img.save(name)

In [2]:
from tqdm import tqdm
# from p_tqdm import p_map
# doesn't work well

In [3]:
for p in tqdm(glob.glob('PCAM/*.*L')):
    try:
        img = read_pds(p)
        if img.shape == (1728, 2352):
            img = debayer_img(img)
        elif img.shape != (864, 1176):
            print(f'{p} has an unexpected dim')
        img = stretch_img(img)
        export_img(f"{p}.png", img)
    except:
        print(p)

100%|██████████| 2564/2564 [25:40<00:00,  1.66it/s] 


In [4]:
for p in tqdm(glob.glob('TCAM/*.*L')):
    try:
        img = read_pds(p)
        export_img(f"{p}.png", img)
    except:
        print(p)

100%|██████████| 240/240 [04:30<00:00,  1.13s/it]


In [3]:
import os
def extract_file_info(p):
    cam,_,ser = p.split('-')  # CE4_GRAS_TCAM I 002_SCI_N_20190106033401_20190106033401_0004_A.2CL.png
    no,_,_,ts,te,ob,ver = ser.split('_') # 002 N N 20190106033401 20190106033401 0004 A.2CL.png
    ver = ver.split('.')[0] # A.2CL.png -> A
    cam = cam.split('_')[2] # TCAM
    return cam,no,ts,te,ob,ver
    
def make_dir_if_n_exist(d):
    if not os.path.exists(d):
        os.makedirs(d)

def make_new_name(no, ts, cam):
    return f"{no}-{ts}-{cam}.png"

In [9]:
observations = {}
CAM_DIR = 'TCAM'
for p in glob.glob(f'{CAM_DIR}/*.png'):
    cam,no,ts,te,ob,ver = extract_file_info(p)
    if ob in observations:
        observations[ob].append(p)
    else:
        observations[ob] = [p]
        
for k in observations.keys():
    d = f"{CAM_DIR}/{k}"
    make_dir_if_n_exist(d)
    for p in observations[k]:
        cam,no,ts,te,ob,ver = extract_file_info(p)
        name = make_new_name(no, ts, cam)
        os.symlink(f"../../{p}", f"{d}/{name}")

In [10]:
observations = {}
CAM_DIR = 'PCAM'
for p in glob.glob(f'{CAM_DIR}/*.png'):
    cam,no,ts,te,ob,ver = extract_file_info(p)
    if ob in observations:
        observations[ob].append(p)
    else:
        observations[ob] = [p]
        
for k in observations.keys():
    d = f"{CAM_DIR}/{k}"
    make_dir_if_n_exist(d)
    for p in observations[k]:
        cam,no,ts,te,ob,ver = extract_file_info(p)
        name = make_new_name(no, ts, cam)
        os.symlink(p, f"{d}/{name}")